# 🎨 WIWS ComfyUI Colab (Pure Edition)

**Created by WIWS_MPA_C00_AI_DNA_Core_Autonomous_System**

**[특징]**
1. **초경량/고속**: 불필요한 자동 설치를 모두 제거했습니다.
2. **순정 상태**: ComfyUI와 필수 매니저(Manager)만 설치됩니다.
3. **자유도 100%**: 모델, 로라, 커스텀 노드는 사용자가 원할 때 직접 설치합니다.
4. **영구 저장**: 구글 드라이브와 연동되어 파일은 계속 유지됩니다.

**사용법: 순서대로 재생 버튼(▶)을 누르세요.**

### 1. 구글 드라이브 연동 (필수)
팝업이 뜨면 '허용'을 눌러주세요.

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

# 기본 폴더 구조 생성 (파일은 다운로드 안 함)
base_path = '/content/drive/MyDrive/WIWS_SD_Forge'
if not os.path.exists(base_path):
    base_path = '/content/drive/MyDrive/WIWS_ComfyUI'
    os.makedirs(base_path, exist_ok=True)

os.makedirs(f'{base_path}/outputs', exist_ok=True)
os.makedirs(f'{base_path}/models/checkpoints', exist_ok=True)
os.makedirs(f'{base_path}/models/loras', exist_ok=True)
os.makedirs(f'{base_path}/models/embeddings', exist_ok=True)
os.makedirs(f'{base_path}/models/vae', exist_ok=True)
os.makedirs(f'{base_path}/models/upscale_models', exist_ok=True)
os.makedirs(f'{base_path}/custom_nodes', exist_ok=True)

print(f"✅ 구글 드라이브 연결 완료! 기본 경로: {base_path}")

### 2. ComfyUI 설치 및 실행 (Basic)
설치가 완료되면 `trycloudflare.com` 링크를 클릭하세요.

In [ ]:
import os

%cd /content

if not os.path.exists('/content/ComfyUI'):
    !git clone https://github.com/comfyanonymous/ComfyUI

%cd /content/ComfyUI

# 구글 드라이브 폴더 연결 (Symlinks)
!rm -rf /content/ComfyUI/models/checkpoints
!ln -s "{base_path}/models/checkpoints" /content/ComfyUI/models/checkpoints
!rm -rf /content/ComfyUI/models/loras
!ln -s "{base_path}/models/loras" /content/ComfyUI/models/loras
!rm -rf /content/ComfyUI/models/embeddings
!ln -s "{base_path}/models/embeddings" /content/ComfyUI/models/embeddings
!rm -rf /content/ComfyUI/output
!ln -s "{base_path}/outputs" /content/ComfyUI/output
!rm -rf /content/ComfyUI/custom_nodes
!ln -s "{base_path}/custom_nodes" /content/ComfyUI/custom_nodes

# 업스케일 모델 폴더 연결
!rm -rf /content/ComfyUI/models/upscale_models
!ln -s "{base_path}/models/upscale_models" /content/ComfyUI/models/upscale_models

# 필수: ComfyUI-Manager 하나만 설치 (이게 있어야 다른걸 쉽게 설치함)
if not os.path.exists(f'{base_path}/custom_nodes/ComfyUI-Manager'):
    print("⬇️ ComfyUI-Manager 설치 중...")
    !git clone https://github.com/ltdrdata/ComfyUI-Manager.git "{base_path}/custom_nodes/ComfyUI-Manager"

# 필수 패키지 설치
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121

# Cloudflared 설치
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

# 실행
import subprocess
import threading
import time
import socket

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared...")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("\n\033[92m🔥 접속 링크 (클릭하세요): " + l[l.find("http"):], end="\033[0m\n")

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server